In [32]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

DS_2 = pd.read_csv("ds-boot-1.csv", "\t")
DS_1 = pd.read_csv("ds-boot-2.csv", "\t")

features = DS_1.values[:, 1:29]
targets = DS_1.values[:, 29:].T

## Task 1

In [34]:
def bootstrap(sample, B=1000, s_f=np.mean):
    '''simple bootstrap'''
    gen = np.random.choice(sample, size=(B, len(sample)))
    estimations = s_f(gen, axis=1)
    est_mean = estimations.mean()
    error = (((estimations - est_mean) ** 2).sum() / (B - 1)) ** 0.5
    return est_mean, error

for i in range(1, 6):
    est_mean, error = bootstrap(targets[i - 1], s_f=np.mean)
    print('y' + repr(i), ': mean:', est_mean, ', error:', error)
    est_mean, error = bootstrap(targets[i - 1], s_f=np.median)
    print('y' + repr(i), ': median:', est_mean, ', error:', error)


y1 : mean: 4.016275 , error: 0.454842149274
y1 : median: 4.02225 , error: 0.608841626062
y2 : mean: 5.513125 , error: 0.552779640504
y2 : median: 6.198375 , error: 0.67062487435
y3 : mean: 7.29035725 , error: 0.455078784811
y3 : median: 8.31187 , error: 0.28896124602
y4 : mean: 1.52925675 , error: 0.109911811801
y4 : median: 1.8607 , error: 0.127363579886
y5 : mean: 5.621134 , error: 0.448524820449
y5 : median: 6.181755 , error: 0.44881045643


In [35]:
def lr(A, b):
    '''Linear regression'''
    model = LinearRegression()
    model.fit(A, b)
    return model.coef_, model.intercept_

In [41]:
def bootstrap_ci(A, b, B=1000):
    original_lr = lr(A, b)
    bootstrap_cfs = []
    n = A.shape[0]
    for _ in range(B):
        idx = np.random.randint(n, size=n)
        coef, inter = lr(A[idx], b[idx])
        arr = np.append(coef, inter)
        bootstrap_cfs.append(arr)
    bootstrap_cfs = np.array(bootstrap_cfs)
    
    deltas = original_lr - bootstrap_cfs
    deltas_l, deltas_r = np.percentile(deltas, [2.5, 97.5], axis = 0)
    l_bound = original_lr - deltas_r
    r_bound = original_lr - deltas_l
    return original_lr, l_bound, r_bound

for i in range(1, 6):
    original_lr, l_bound, r_bound = bootstrap_ci(features, targets[i - 1])
#     print('y' + repr(i), ': mean:', est_mean, ', error:', error)


ValueError: operands could not be broadcast together with shapes (2,) (1000,29) 